## Loading Libraries

In [1]:
import pandas as pd
import numpy as np
import torch

## Transforming Data

### Loading Dataset

In [2]:
df = pd.read_csv('../data/processed/train_sample_processed.csv')

In [3]:
df.head(4)

,Unnamed: 0,ReputationAtPostCreation,OwnerUndeletedAnswerCountAtPostTime,Title,BodyMarkdown,DaysTillPosting,UnrecognizedTags,OpenStatus,Tag1,Tag2,Tag3,Tag4,Tag5
0,0,1,2,For Mongodb is it better to reference an objec...,I am building a corpus of indexed sentences in...,243,5,open,NaN,NaN,NaN,NaN,NaN
1,1,192,24,How to insert schemalocation in a xml document...,i create a xml document with JAXP and search a...,122,5,open,NaN,NaN,NaN,NaN,NaN
2,2,1,0,Too many lookup tables,What are the adverse effects of having too man...,0,5,open,27.0,NaN,NaN,NaN,NaN
3,3,4,1,What is this PHP code in VB.net,I am looking for the vb.net equivalent of this...,258,5,too localized,0.0,38.0,NaN,NaN,NaN


### Encoding Tags

In [4]:
selected_tags = np.load('../data/processed/selected_tags.npy')

In [5]:
len(selected_tags)

100

In [6]:

def encodeTags(dataframe):
  tags_columns = ['Tag1', 'Tag2', 'Tag3', 'Tag4', 'Tag5']
  encoded = pd.get_dummies(dataframe['Tag1'], columns=['Tag1'], prefix='tag_')

  for i in range(1, 4):
    
    col_name = tags_columns[i]
    dummies = pd.get_dummies(dataframe[col_name], columns=[col_name], prefix='tag_')
    encoded |= dummies
    # print(i, dummies['tag__86.0'], encoded['tag__86.0'])
    
  return encoded

In [9]:
encoded = encodeTags(df)

# Check if there are on NaNs left
print(encoded.isna().sum().values)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### Encoding Title and BodyMarkdown

### Encoding OpenStatus

## Creating PyTorch Dataset

In [30]:
from torch.utils.data.dataset import Dataset

class SingleExample:
  def __init__(self, tags_onehot_a):
    self.tags_onehot = tags_onehot_a
    # self.tags_onehot = []
    

class CustomDataset(Dataset):
  def __init__(self, df):
    self.tags_onehot = []
    self.examples = []

    # Encoding tags
    encoded = encodeTags(df)
    res = torch.from_numpy(encoded.to_numpy())
    self.tags_onehot = res
    
    # 
    
  def __len__(self):
    return len(self.examples)

  def __getitem__(self, i):
    # We’ll pad at the batch level.
    
    return SingleExample(self.tags_onehot[i])
    # return torch.tensor()

In [33]:
# Create the train dataset
train_dataset = CustomDataset(df)

train_dataset[0]